In [ ]:
# required to run in google colab
!pip install qiskit qiskit_machine_learning qiskit_ibm_runtime

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options
from qiskit import BasicAer, QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit.visualization import circuit_drawer
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.algorithms import QSVC
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.datasets import make_blobs
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd

In [3]:
# you first need to upload the fault.txt and faultlabel.txt files by clicking the file icon on the left and uploading
X = pd.read_csv("fault.txt", sep='\t')
y = pd.read_csv("faultlabel.txt", sep='\t')
X = X.to_numpy()
y = y.to_numpy()
X, y = shuffle(X, y)
X = X[:200]
y = y[:200]
y = list(map(np.argmax, y)) # convert one hot outputs to binary labelling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [4]:
feature_map = ZZFeatureMap(9, reps=2)
kernel = FidelityQuantumKernel(feature_map=feature_map)

In [5]:
# you can find your api token on the home page of ibm quantum right after you log in
# uncomment everything below to run on ibm quantum simulator or leave it commented to simulate locally

# token="PASTE IBM QUANTUM API TOKEN HERE"
# service = QiskitRuntimeService(channel="ibm_quantum", instance="ibm-q-asu/main/spanias", token=token)
# options = Options(optimization_level=1)
# session = Session(service=service, backend="ibmq_qasm_simulator")
# sampler = Sampler(session=session, options=options)
# fidelity = ComputeUncompute(sampler=sampler)
# kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [ ]:
svm = QSVC(quantum_kernel=kernel)
svm.fit(X_train, y_train)

In [ ]:
outputs = svm.predict(X_test)
cm = confusion_matrix(y_test, outputs)
disp = ConfusionMatrixDisplay(cm)
disp.plot()